In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('graphs_with_petal_3.csv')
df['y'] = np.array(df['class'] == 'complex', dtype=np.float64)

In [ ]:
df

In [ ]:
X = df.iloc[:,2:-1].to_numpy()
y = df['y'].to_numpy(dtype=np.float64)

sc = StandardScaler()
X = sc.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=17, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=100, batch_size=10);

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# complex = 1, synthetic = 0

x = X_test
Y = y_test

pred = [round(i[0]) for i in model.predict(x)]
synC = 0;
synF = 0;
comC = 0;
comF = 0;

for i in range(len(Y)):
    if Y[i] == 0 and pred[i] == 0:
        synC += 1
    elif Y[i] == 0 and pred[i] == 1:
        synF += 1
    elif Y[i] == 1 and pred[i] == 1:
        comC += 1
    elif Y[i] == 1 and pred[i] == 0:
        comF += 1
        
print("Pred/Act|Syn|Com\nSyn\t|",synC, synF, "\nCom\t|", comF, comC)

In [ ]:
# model and sc have to be in RAM
# function to predict an instance by its name
model.finalize_state()
def predictInstance(i):
    ins = df[df['instance'] == i].iloc[:,2:-1].to_numpy(dtype=np.float64)
    ins = sc.transform(ins)
    val = model.predict(ins, callbacks=None, verbose=False);
    return val[0][0]

In [ ]:
# All instances that have been predicted worngly
wrong = {}
for r in tqdm(df.iterrows(), total=len(df)):
    data = r[1]
    name = data['instance']
    y = data['y']
    pred = predictInstance(name);
    if round(pred) != round(y):
        wrong[name] = pred
        
sorted(wrong)